In [1]:
import pandas as pd
df = pd.read_json("data/anatole_data_test.json")
df.head()

#message_nohist= [''' 'variant': 'NT', 'ante_trimming_status': True, 'hand': 78, 'players': ['p1', 'p2', 'p3', 'p4', 'p5', 'p6'], 'player': 'p5', 'antes': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'blinds_or_straddles': [50.0, 100.0, 0.0, 0.0, 0.0, 0.0], 'min_bet': 100, 'starting_stacks': [10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0], 'actions': "['d dh p1 ****', 'd dh p2 ****', 'd dh p3 ****', 'd dh p4 ****', 'd dh p5 Ad4d', 'd dh p6 ****', 'p3 cbr 225', 'p4 f', 'p5 " >''']
#message_hist=[''' "variant":"NT","ante_trimming_status":true,"hand":21,"players":["p1","p2","p3","p4","p5","p6"],"player":"p2","antes":[0.0,0.0,0.0,0.0,0.0,0.0],"blinds_or_straddles":[50.0,100.0,0.0,0.0,0.0,0.0],"min_bet":100,"history":{"hand 0":["d dh p1 ****","d dh p2 ****","d dh p3 ****","d dh p4 ****","d dh p5 KdAd","d dh p6 ****","p3 f","p4 f","p5 cbr 210","p6 f","p1 cc","p2 f","d db TdKc9d","p1 cc","p5 cbr 260","p1 f"],"hand 1":["d dh p1 ****","d dh p2 ****","d dh p3 ****","d dh p4 Js7d","d dh p5 ****","d dh p6 ****","p3 f","p4 f","p5 f","p6 f","p1 f"],"hand 2":["d dh p1 ****","d dh p2 ****","d dh p3 9h3d","d dh p4 ****","d dh p5 ****","d dh p6 ****","p3 f","p4 cbr 250","p5 f","p6 f","p1 f","p2 f"],"hand 3":["d dh p1 ****","d dh p2 6c8d","d dh p3 ****","d dh p4 ****","d dh p5 ****","d dh p6 ****","p3 f","p4 f","p5 f","p6 f","p1 cbr 300","p2 cc","d db TsAcKh","p1 cbr 450","p2 f"],"hand 4":["d dh p1 AsJc","d dh p2 ****","d dh p3 ****","d dh p4 ****","d dh p5 ****","d dh p6 ****","p3 f","p4 f","p5 f","p6 cbr 287","p1 cbr 1150","p2 f","p6 cbr 2425","p1 f"],"hand 5":["d dh p1 ****","d dh p2 ****","d dh p3 ****","d dh p4 ****","d dh p5 ****","d dh p6 ****","p3 f","p4 f","p5 cbr 225","p6 f","p1 f","p2 f"],"hand 6":["d dh p1 ****","d dh p2 ****","d dh p3 ****","d dh p4 ****","d dh p5 7h2s","d dh p6 ****","p3 f","p4 f","p5 f","p6 f","p1 f"],"hand 7":["d dh p1 ****","d dh p2 ****","d dh p3 ****","d dh p4 Th4c","d dh p5 ****","d dh p6 ****","p3 f","p4 f","p5 cbr 225","p6 f","p1 f","p2 f"],"hand 8":["d dh p1 ****","d dh p2 ****","d dh p3 8d6c","d dh p4 ****","d dh p5 ****","d dh p6 ****","p3 f","p4 f","p5 cbr 200","p6 f","p1 f","p2 f"]},"starting_stacks":[10000.0,10000.0,10000.0,10000.0,10000.0,10000.0],"actions":"['d dh p1 ****', 'd dh p2 2hAs', 'd dh p3 ****', 'd dh p4 ****', 'd dh p5 ****', 'd dh p6 ****', 'p3 f', 'p4 f', 'p5 f', 'p6 f', 'p1 cbr 300', 'p2 cc', 'd db 3cKcJc', 'p1 cc', 'p2 cc', 'd db Ks', 'p1 cbr 450', 'p2 " >''']


,instruction,input,output
0,Game ID: 793109047 0.10/0.25 (PRR) Ekek - 5 (H...,,checks
1,Game ID: 791882473 0.10/0.25 (PRR) Ekek - 13 (...,,folds
2,Game ID: 793955493 0.25/0.50 (PRR) Gorgons (Ca...,,folds
3,Game ID: 789358065 1/2 (PRR) Longana - 3 (Hold...,,folds
4,Game ID: 786933996 0.25/0.50 (PRR) Gorgons - 7...,,folds


In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use CUDA if available
from tqdm import tqdm
llm_response = []
mistral_reponse=[]
path="/home/avakili/finetuning/output/outpok1/final_output" #dataset julien sans historique
path="/home/avakili/finetuning/output/outpok2/final_output" #dataset julien avec historique
path="/dataset/nlp/DatasetTestCyberJarvis/outpok4/final_output" #dataset anatole ajsuté
path="mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(path, cache_dir="/dataset/nlp/Huggingface_cache")
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(path)
for k in tqdm(range(len(df))): 
    message_so=[df['instruction'].iloc[k]]    
    encoded = tokenizer(message_so,  return_tensors="pt").to(device)
    output = model.generate(**encoded,do_sample=False, temperature=0., max_new_tokens=12)
    decoded = tokenizer.decode(output[0][encoded["input_ids"].shape[-1]:], skip_special_tokens=True)
    mistral_reponse.append(decoded)

  0%|          | 0/3454 [00:00<?, ?it/s]/home/avakili/finetuning/ftvenv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 3454/3454 [15:24<00:00,  3.74it/s]


In [29]:
%run metrics.ipynb
waited_output=df["output"].to_list()

In [44]:
mistral_reponse= [mistral_reponse[k].split()[0] for k in range(len(mistral_reponse))]


In [45]:
print(compare_to_test(waited_output, mistral_reponse))
print('nombre de coup légal:' , legal_actions(mistral_reponse), len(mistral_reponse))

        bets  calls  raises  allin  checks  folds  caps
bets      26      0       0      0      10      0     0
calls      8     66      23      0       5      0     0
raises    39     18      23      0       0      0     0
allin      0      7       1      0       1      0     0
checks    49      0       0      0     111      0     0
folds    808    856     617      0      54     87     0
caps       0      1       0      0       0      0     0
nombre de coup légal: 2810 3454


In [23]:
wrong_indicies = find_misclassifications(waited_output, llm_response)
n=wrong_indicies[3]
print(df['instruction'].iloc[n], "\nwaited_output", waited_output[n], "\nllm response:", llm_response[n] )

Game ID: 808774852 0.25/0.50 (PRR) Gorgons (Cap) - 2 (Hold'em)
Seat 1 is the button
Seat 1: nowhere2be (46.20).
Seat 2: imperceptible (9.51).
Seat 3: elec_365 (63.32).
Seat 4: Jui Kuen (50).
Seat 5: IlxxxlI (34).
Player imperceptible has small blind (0.25)
Player elec_365 has big blind (0.50)
Player IlxxxlI received card: [4c]
Player IlxxxlI received card: [4d]
Player Jui Kuen folds
Player IlxxxlI 
waited_output raises (1) 
llm response: folds


#Sauvegarde 

In [46]:
import csv

# Saving
my_list = mistral_reponse
with open('Mistral7b_response_test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(my_list)

# Piepline pour la range pre flop